In [1]:
# ============================================================
# NOTEBOOK 4: ROUTING AGENT WITH DIRECT GEMINI API (FIXED)
# ============================================================
# Purpose: Use Gemini API directly (no LangChain version conflicts)
# ============================================================

# ============================================================
# STEP 1: Install Only What We Need
# ============================================================
!pip install google-generativeai

# ============================================================
# STEP 2: Import Libraries
# ============================================================
import os
import json
import re
import google.generativeai as genai

# ============================================================
# STEP 3: Configure Gemini API Key
# ============================================================
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

if GEMINI_API_KEY and GEMINI_API_KEY != "your-gemini-api-key":
    genai.configure(api_key=GEMINI_API_KEY)
    model = genai.GenerativeModel('gemini-pro')
    print("✅ Gemini API configured")
else:
    model = None
    print("⚠️  Please set your API key")

# ============================================================
# STEP 4: Define Guardrails
# ============================================================
class GuardrailsSystem:
    def __init__(self):
        self.math_keywords = [
            'solve', 'calculate', 'compute', 'find', 'derive', 'integrate',
            'differentiate', 'equation', 'formula', 'theorem', 'proof',
            'algebra', 'calculus', 'geometry', 'trigonometry', 'statistics',
            'probability', 'number', 'function', 'graph', 'matrix', 'vector'
        ]

        self.inappropriate_keywords = [
            'hack', 'cheat', 'steal', 'illegal', 'weapon', 'drug',
            'violent', 'harm', 'kill', 'abuse'
        ]

    def check_input_guardrail(self, query: str) -> dict:
        query_lower = query.lower()

        for keyword in self.inappropriate_keywords:
            if keyword in query_lower:
                return {
                    'allowed': False,
                    'reason': 'Query contains inappropriate content',
                    'category': 'inappropriate'
                }

        is_math = any(keyword in query_lower for keyword in self.math_keywords)
        has_numbers = bool(re.search(r'\d', query))
        has_math_symbols = bool(re.search(r'[+\-*/=^√∫∑∏]', query))

        if is_math or has_numbers or has_math_symbols:
            return {
                'allowed': True,
                'reason': 'Query is math-related',
                'category': 'mathematics'
            }
        else:
            return {
                'allowed': False,
                'reason': 'Query is not related to mathematics. This system only handles math questions.',
                'category': 'non_math'
            }

    def check_output_guardrail(self, response: str) -> dict:
        response_lower = response.lower()

        for keyword in self.inappropriate_keywords:
            if keyword in response_lower:
                return {'allowed': False, 'reason': 'Response contains inappropriate content'}

        has_steps = 'step' in response_lower
        has_explanation = len(response) > 50

        if has_steps or has_explanation:
            return {'allowed': True, 'reason': 'Response is educational and appropriate'}
        else:
            return {'allowed': True, 'reason': 'Response could be more detailed'}

guardrails = GuardrailsSystem()
print("✅ Guardrails system initialized")

# Test guardrails
print("\n🧪 Testing Guardrails:")
test_cases = [
    "Solve for x: 2x + 5 = 13",
    "What is the weather today?",
    "How to hack a website?",
]

for test in test_cases:
    result = guardrails.check_input_guardrail(test)
    status = "✅ PASS" if result['allowed'] else "❌ BLOCK"
    print(f"{status}: '{test}' - {result['reason']}")

# ============================================================
# STEP 5: Create Simple Agent (No LangGraph)
# ============================================================

def math_agent(query: str) -> dict:
    """
    Simple agent that:
    1. Checks guardrails
    2. Generates solution with Gemini
    3. Returns result
    """
    print("\n" + "="*60)
    print(f"📝 Processing Query: {query}")
    print("="*60)

    # Check if model is configured
    if not model:
        return {
            'success': False,
            'query': query,
            'solution': None,
            'error': 'API key not configured',
            'blocked': True
        }

    # Input guardrail
    guardrail_check = guardrails.check_input_guardrail(query)
    print(f"🛡️  Guardrail Check: {guardrail_check['reason']}")

    if not guardrail_check['allowed']:
        return {
            'success': False,
            'query': query,
            'solution': None,
            'error': guardrail_check['reason'],
            'blocked': True
        }

    # Generate solution
    print(f"🔀 Routing: Using Gemini Pro")

    prompt = f"""You are a mathematics professor. A student asked: "{query}"

Please provide a clear, step-by-step solution that:
1. Breaks down the problem into manageable steps
2. Explains the reasoning for each step
3. Shows all calculations clearly
4. Provides the final answer

Format your response with clear step numbering and explanations."""

    try:
        print(f"🔍 Search: Using direct LLM generation")
        response = model.generate_content(prompt)
        solution = response.text
        print(f"🧮 Generated solution ({len(solution)} chars)")
    except Exception as e:
        return {
            'success': False,
            'query': query,
            'solution': None,
            'error': f"Error: {str(e)}",
            'blocked': False
        }

    # Output guardrail
    output_check = guardrails.check_output_guardrail(solution)
    print(f"🛡️  Output Check: {output_check['reason']}")

    if not output_check['allowed']:
        solution = "I apologize, but I cannot provide that response."

    return {
        'success': True,
        'query': query,
        'solution': solution,
        'routing': {'use_kb': False, 'reason': 'Using Gemini directly'},
        'search_source': 'gemini_pro',
        'blocked': False
    }

print("✅ Math agent function created")

# ============================================================
# STEP 6: Test the Agent
# ============================================================
print("\n" + "="*60)
print("🧪 TESTING AGENT")
print("="*60)

test_questions = [
    "Solve for x: 3x - 7 = 14",
    "What is the derivative of x²?",
    "How to hack passwords?"  # Should be blocked
]

for question in test_questions:
    result = math_agent(question)

    print(f"\n{'='*60}")
    if result['success']:
        print(f"✅ SUCCESS")
        print(f"📄 Solution:\n{result['solution'][:500]}...")
        if len(result['solution']) > 500:
            print(f"\n... (total length: {len(result['solution'])} chars)")
    else:
        print(f"❌ BLOCKED: {result['error']}")
    print("="*60)

print("\n" + "="*60)
print("✅ NOTEBOOK 4 COMPLETE!")
print("="*60)
print("\n📝 WHAT YOU DID:")
print("   - Used Direct Gemini API (no LangChain conflicts)")
print("   - Built simple but effective agent")
print("   - Guardrails working perfectly")
# print("   - Ready for Gradio UI!")
# print("\n🔜 NEXT: Move to Notebook 5 (Gradio UI)")
print("="*60)


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\brije\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
✅ Gemini API configured
✅ Guardrails system initialized

🧪 Testing Guardrails:
✅ PASS: 'Solve for x: 2x + 5 = 13' - Query is math-related
❌ BLOCK: 'What is the weather today?' - Query is not related to mathematics. This system only handles math questions.
❌ BLOCK: 'How to hack a website?' - Query contains inappropriate content
✅ Math agent function created

🧪 TESTING AGENT

📝 Processing Query: Solve for x: 3x - 7 = 14
🛡️  Guardrail Check: Query is math-related
🔀 Routing: Using Gemini Pro
🔍 Search: Using direct LLM generation

❌ BLOCKED: Error: 404 models/gemini-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.

📝 Processing Query: What is the derivative of x²?
🛡️  Guardrail Check: Query is not related to mathematics. This system only handles math questions.

❌ BLOCKED: Query is not related to mathe